In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# # Set the desired working directory
# new_working_directory = '/content/drive/My Drive'

# # Change the current working directory
# os.chdir(new_working_directory)

# Import dataset

In [1]:
import pandas as pd
import numpy as np
from scipy import special
from scipy import stats
import matplotlib.pyplot as plt
import math
import itertools
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)

In [3]:
df = pd.read_csv('2.22 combined dataset of stock data and balance sheet data.csv')
df['isin'].nunique()

199

# Change fiscal_quarter

In [4]:
series_list = []
for i in range(1,(df[df['company_name'] == df['company_name'].unique()[0]].shape[0])+1):
  series_list.append(i)

In [5]:
temp = pd.DataFrame({'fiscal_quarter': series_list * (len(df['company_name'].unique()))})
df['fiscal_quarter'] = temp['fiscal_quarter']

In [6]:
len(df['company_name'].unique())

199

In [7]:
temp['fiscal_quarter'].shape

(14328,)

In [8]:
df["datadate"] = pd.to_datetime(df["datadate"])

In [9]:
df.head(5)

,datadate,key,fiscal_quarter,fiscal_year,total_assets,total_liabilities,isin,company_name,gic_industries,iso_country_code,industry_name,is_finance,Open,High,Low,Close,Volume
0,2006-03-31,212935.0,1,2006,74.4419,27.4910,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.185858,0.202586,0.170246,0.185858,8522223.00
1,2006-06-30,212935.0,2,2006,77.1485,32.3303,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.185858,0.187717,0.131959,0.141996,8754235.25
2,2006-09-30,212935.0,3,2006,89.3972,36.4684,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.142368,0.148687,0.133818,0.146456,6738297.00
3,2006-12-31,212935.0,4,2006,77.2313,23.2864,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.147943,0.174707,0.144970,0.162440,8879853.35
4,2007-03-31,212935.0,5,2007,83.2715,29.5663,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.160571,0.179738,0.152226,0.158462,8679271.40


# Define functions used to implement Zhou's Model using emperical data from company balance sheet and stock returns

In [32]:
### These are function producing the estimated parameters used in the Zhou(2001) model
def get_companies():
  return df['company_name'].unique()

def get_industries():
  return df['industry_name'].unique()

def get_date():
  return df['datadate'].unique()

def get_num_quarter():
  return len(df['fiscal_quarter'].value_counts())

def get_country_codes():
  return df['iso_country_code'].unique()

def get_companies_from_industry(industry):
  return df[df['industry_name'] == industry]['company_name'].unique()

def get_companies_from_country(country):
  return df[df['iso_country_code'] == country]['company_name'].unique()

def get_finance_companies():
  return df[df['is_finance']==True]['company_name'].unique().tolist()

def get_Vi0(start,company1,company2):
  '''
  This function takes the a start date and two company names to return the starting value of assets
  Output: [v_company1_0,v_company_2_0]
  '''
  v_10 = df[(df['company_name'] == company1)&(df['datadate'] == start)]['total_assets'].item()
  v_20 = df[(df['company_name'] == company2)&(df['datadate'] == start)]['total_assets'].item()
  return v_10,v_20

def get_Ki(start,company1,company2):
  '''
  This function takes the a start date and two company names to return the starting value of Ki := short-term liability + 50% long-term liability
                                                                                                 = total liability -50% long-term liability
  Output: k_company1,k_company_2
  '''
  total_liability_1 = df.loc[(df['company_name'] == company1)&(df['datadate'] == start),['total_liabilities']].iloc[0,0]
  total_liability_2 = df.loc[(df['company_name'] == company2)&(df['datadate'] == start),['total_liabilities']].iloc[0,0]
  # long_term_liability_1 = df.loc[(df['company_name'] == company1)&(df['datadate'] == start),['long_term_liabilities']].iloc[0,0]
  # long_term_liability_2 = df.loc[(df['company_name'] == company2)&(df['datadate'] == start),['long_term_liabilities']].iloc[0,0]
  return  0.75*total_liability_1,0.75*total_liability_2

def get_t(start,end):
  '''
  This function takes the start date and end date of a time horizon and return t
  in Zhou(2001) formula
  Output: t(in years)
  '''
  start_idx = np.where(df['datadate'] == start)[0][0]
  end_idx = np.where(df['datadate'] == end)[0][0]
  return (end_idx - start_idx)/4

def get_sigma_i(start,end,company):
  '''
  This function returns the estimated asset volatility of a firm
  '''
  close_list = df[(df['datadate'] >= start) & (df['datadate'] <= end)&(df['company_name'] == company)]['Close']
  close_pct = close_list.pct_change()[1:]
  #print(close_list)
  return close_pct.std()


def get_rho(start,end,companies,column,company_1 = None,company_2 = None):
  '''
  This function takes the start date and end date of a time frame and returns the correlation matrix of asset value in a given time horizon
  Note companies is a numpy array containing the name of companies
  Output: A list object:[whole correlation matrix, corr(companyname1,companyname2)] if company_1 and company_2 specified
          the whole correlation matrix otherwise
  '''
  diff_matrix = np.zeros((int(get_t(start,end)*4),len(companies)))
  for i in [i for i in range(0,len(companies))]:
    col_val = np.diff(np.log(df[(df['datadate'] >= start) & (df['datadate'] <= end)&(df['company_name'] == companies[i])][column]))
    diff_matrix [:,i]= col_val
  diff_matrix = pd.DataFrame(diff_matrix)
  corr_df  = diff_matrix.corr()
  if company_1 == None and company_2 == None:
    return corr_df
  else:
    company_1_idx = companies.tolist().index(company_1)
    company_2_idx = companies.tolist().index(company_2)
    output = corr_df[company_1_idx][company_2_idx]
    return output

In [11]:
for i in range(len(get_country_codes())):
  print(get_country_codes()[i], df[df['iso_country_code'] == get_country_codes()[i]]['company_name'].nunique())

ARG 26
PER 1
CUW 1
BRA 54
CHL 51
COL 10
JAM 1
CYM 1
MEX 48
PAN 2
TTO 1
VGB 3


In [12]:
df[df['iso_country_code']== 'COL']

,datadate,key,fiscal_quarter,fiscal_year,total_assets,total_liabilities,isin,company_name,gic_industries,iso_country_code,industry_name,is_finance,Open,High,Low,Close,Volume
9576,2006-03-31,205182.0,1,2006,1.820810e+05,38980.180000,COC07PA00027,MINEROS SA,151040.0,COL,Metals & Mining,False,0.593468,0.784226,0.572273,0.741836,2.208212e+07
9577,2006-06-30,205182.0,2,2006,1.820810e+05,38980.180000,COC07PA00027,MINEROS SA,151040.0,COL,Metals & Mining,False,0.746075,1.038570,0.678250,0.792704,4.058738e+07
9578,2006-09-30,205182.0,3,2006,1.820810e+05,38980.180000,COC07PA00027,MINEROS SA,151040.0,COL,Metals & Mining,False,0.792704,0.932593,0.763031,0.801182,5.809771e+06
9579,2006-12-31,205182.0,4,2006,1.820810e+05,38980.180000,COC07PA00027,MINEROS SA,151040.0,COL,Metals & Mining,False,0.792704,1.059765,0.775748,0.996179,7.799280e+06
9580,2007-03-31,205182.0,5,2007,1.841106e+05,35910.398356,COC07PA00027,MINEROS SA,151040.0,COL,Metals & Mining,False,1.131413,1.169929,1.015865,1.145857,1.009397e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10291,2022-12-31,292768.0,68,2022,2.580020e+06,584876.000000,COW19PA00020,PROTECCION SA,402030.0,COL,Capital Markets,True,28.158742,28.158742,28.158742,28.158742,3.225492e+03
10292,2023-03-31,292768.0,69,2023,2.698882e+06,593777.147000,COW19PA00020,PROTECCION SA,402030.0,COL,Capital Markets,True,27.556860,27.556860,18.371240,18.371240,2.909312e+03
10293,2023-06-30,292768.0,70,2023,2.784071e+06,597825.375000,COW19PA00020,PROTECCION SA,402030.0,COL,Capital Markets,True,18.371240,18.371240,18.371240,18.371240,2.173139e+03
10294,2023-09-30,292768.0,71,2023,2.786187e+06,613564.366000,COW19PA00020,PROTECCION SA,402030.0,COL,Capital Markets,True,18.371240,18.371240,9.185620,9.185620,1.752061e+03


In [13]:
get_Vi0('2006-03-31','PAMPA ENERGIA SA','TELECOM ARGENTINA'	)

(36.4952, 8916.0)

In [14]:
get_Ki('2006-03-31','PAMPA ENERGIA SA','TELECOM ARGENTINA')

(6.87285, 5249.25)

In [15]:

get_t('2006-03-31','2021-03-31')

15.0

In [16]:
# s = get_rho('2018-03-31','2023-03-31',get_companies(),'Close').stack()
# l = list(filter( lambda x: x[0]!= x[1],s[s>0.99].index.tolist()))
# l

In [17]:
get_rho(get_date()[0],get_date()[3],get_companies(),'Close').head(3)

,0,1,2,3,4,5,6,7,8,9,...,189,190,191,192,193,194,195,196,197,198
0,1.000000,-0.633200,0.677572,0.904816,0.989767,0.721817,0.835530,0.495626,0.901614,-0.559971,...,0.838838,-0.315740,0.982783,0.674592,0.944351,0.904763,0.945420,0.653276,0.565793,0.986711
1,-0.633200,1.000000,0.140197,-0.243364,-0.516281,0.078610,-0.103794,0.358406,-0.236120,0.995832,...,-0.952497,0.934322,-0.765304,0.144200,-0.343368,-0.902549,-0.850847,-0.999656,-0.996450,-0.499024
2,0.677572,0.140197,1.000000,0.926237,0.775580,0.998081,0.970224,0.974593,0.929024,0.229915,...,0.168003,0.483899,0.530020,0.999992,0.881786,0.299800,0.400939,-0.114188,-0.223054,0.788068


In [29]:
@jit
def get_default_corr_zhou(v1,v2,k1,k2,sigma1,sigma2,rho,t):
  '''
  This function returns default correlation calculated by zhou
  '''
  z1 = (math.log(v1/k1))/sigma1
  z2 = (math.log(v2/k2))/sigma2
  alpha = math.atan((-math.sqrt(1-rho**2))/rho) if rho < 0 else math.pi +  math.atan((-math.sqrt(1-rho**2))/rho)
  theta0 = math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2)) if (z2*math.sqrt(1-rho**2))/(z1-rho*z2) > 0 else math.pi + math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2))
  r0 = z2/math.sin(theta0)
  inf_sum = 0
  for n in [i for i in range(1,100000,2)]:
    bassel_sum = special.iv(0.5*(((n*math.pi)/alpha)+1),(r0**2/(4*t)))+ special.iv(0.5*(((n*math.pi)/alpha)-1),(r0**2/(4*t)))
    inf_sum += 1/n * math.sin((n*math.pi*theta0)/alpha)*bassel_sum
  marginal_prob_1 =  2*stats.norm.cdf(-z1/math.sqrt(t)) if 2*stats.norm.cdf(-z1/math.sqrt(t))< 1 else 0.9
  marginal_prob_2 = 2*stats.norm.cdf(-z2/math.sqrt(t))if 2*stats.norm.cdf(-z2/math.sqrt(t))<1 else 0.9
  var_1 = marginal_prob_1*(1-marginal_prob_1)
  var_2 = marginal_prob_2*(1-marginal_prob_2)
  either_default_prob = 1-(2*r0/(math.sqrt(2*math.pi*t)))*math.exp(-(r0**2/(4*t)))*inf_sum
  default_corr = (marginal_prob_1+marginal_prob_2-either_default_prob-marginal_prob_1*marginal_prob_2)/(math.sqrt(var_1*var_2))
  return default_corr
@jit
def analyse_two_companies_zhou(start,end,companies,company1,company2,prob_sub = 0.9):
  '''
  This function returns the default correlation between 2 companies in a given time horizon by using the
  closed form formula from Zhou(2001)
  prob_sub is the subsituting probability if one of the marginal probability is greater than 1
  '''
  ### get the essential parameter for Zhou's either probability default
  # print(companies.tolist().index(company1),companies.tolist().index(company2))
  v1 = get_Vi0(start,company1,company2)[0]
  v2 = get_Vi0(start,company1,company2)[1]
  k1 = get_Ki(start,company1,company2)[0]
  k2 = get_Ki(start,company1,company2)[1]
  sigma1 = get_sigma_i(start,end,company1)
  sigma2 = get_sigma_i(start,end,company2)
  #print(sigma1,sigma2)
#   print(get_rho(start,end,companies,'Close',company1,company2))
  if (abs(get_rho(start,end,companies,'Close',company1,company2))>=0.99)or(get_rho(start,end,companies,'Close',company1,company2)==0):
    rho = 0.5
  else:
    rho = get_rho(start,end,companies,'Close',company1,company2)
#   print(rho)
  t = get_t(start,end)
  z1 = (math.log(v1/k1))/sigma1
  z2 = (math.log(v2/k2))/sigma2
  alpha = math.atan((-math.sqrt(1-rho**2))/rho) if rho < 0 else math.pi +  math.atan((-math.sqrt(1-rho**2))/rho)
  theta0 = math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2)) if (z2*math.sqrt(1-rho**2))/(z1-rho*z2) > 0 else math.pi + math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2))
  r0 = z2/math.sin(theta0)
  inf_sum = 0
  for n in [i for i in range(1,1000000,2)]:
    bassel_sum = special.iv(0.5*(((n*math.pi)/alpha)+1),(r0**2/(4*t)))+ special.iv(0.5*(((n*math.pi)/alpha)-1),(r0**2/(4*t)))
    inf_sum += 1/n * math.sin((n*math.pi*theta0)/alpha)*bassel_sum
  #print(round(inf_sum,1),round(v1/k1,1),round(v2/k2,1),round(rho,2),t,round(z1,1),round(z2,1),round(alpha,1),round(theta0,1),round(r0,1))
  ### calculate the marginal default probability and correlation
  marginal_prob_1 = 2*stats.norm.cdf(-z1/math.sqrt(t))
  marginal_prob_2 = 2*stats.norm.cdf(-z2/math.sqrt(t))
  ### store the original value if any of the marginal prob >=1
  marginal_prob_1_original = marginal_prob_1 if marginal_prob_1 >=1 else marginal_prob_1
  marginal_prob_2_original = marginal_prob_2 if marginal_prob_2 >=1 else marginal_prob_2
  ### use the prob_sub to calculate the correlation
  marginal_prob_1_cal = prob_sub if marginal_prob_1 >=1 else marginal_prob_1
  marginal_prob_2_cal = prob_sub if marginal_prob_2 >=1 else marginal_prob_2
  var_1 = marginal_prob_1*(1-marginal_prob_1_cal)
  var_2 = marginal_prob_2*(1-marginal_prob_2_cal)
  either_default_prob = 1-(2*r0/(math.sqrt(2*math.pi*t)))*math.exp(-(r0**2/(4*t)))*inf_sum
  ### if the inf_sum returns value NaN just put default correlation to 1.5 and so that it can be ignored in later calculation
#   print(np.isnan(get_default_corr_zhou(v1,v2,k1,k2,sigma1,sigma2,rho,t)))
  default_corr = (marginal_prob_1_cal+marginal_prob_2_cal-either_default_prob-marginal_prob_1_cal*marginal_prob_2_cal)/(math.sqrt(var_1*var_2)) if (np.isnan(get_default_corr_zhou(v1,v2,k1,k2,sigma1,sigma2,rho,t)) == False) and (rho !=0.5) else 1.5
  return default_corr,marginal_prob_1_original,marginal_prob_2_original

@jit
def get_decorr_two_group_companies_zhou(start,end,company_list1,company_list2,prob_sub=0.9,show_non_normal = None):
  companies = get_companies()
  if company_list1 != company_list2:
    combination_list =  list(itertools.product(company_list1,company_list2))
  else:
    combination_list = list(filter(lambda x: x[0]!=x[1],list(itertools.combinations(company_list1,2))))
  # print(len(company_list1),len(company_list2),len(combination_list))
#   print(company_list1)
#   print(company_list2)
#   print(combination_list)
  print(len(combination_list))
  num_pairs = len(combination_list)
  corr_sum = 0
  not_normal_sum = 0
  non_normal_dic = {}
  for (firm1,firm2) in combination_list:
    result = analyse_two_companies_zhou(start,end,companies,firm1,firm2,prob_sub)
    corr = result[0]
    #print(corr_sum)
    prob_default_1 = result[1]
    prob_default_2 = result[2]
    # print(corr,prob_default_1,prob_default_2)
    # print(abs(corr)>1)
#     print(corr,type(corr))
    if (abs(corr) > 1) :
      num_pairs -= 1
#       print(num_pairs)
      corr_sum += 0
      non_normal_dic[corr] = [prob_default_1,prob_default_2]
    else:
      corr_sum += corr
  #print(num_pairs)
  if show_non_normal == 'Yes':
    return non_normal_dic
  else:
    return corr_sum/num_pairs if num_pairs != 0 else None

C:\Users\ZYWCH\AppData\Local\Temp\ipykernel_9556\4156262280.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\ZYWCH\AppData\Local\Temp\ipykernel_9556\4156262280.py:22: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\ZYWCH\AppData\Local\Temp\ipykernel_9556\4156262280.py:72: NumbaDeprecationWarning: The 'nopython' keyword argument was n

# get the country coefficient

In [26]:
from numba import jit
from joblib import Parallel, delayed


In [30]:
@jit(nopython=True)
def get_country_corr(start,end,country,prob_sub=0.9):
  companies = get_companies()
  company_list_financial = df[(df['iso_country_code'] == country)&(df['is_finance'] == True)]['company_name'].unique().tolist()
  company_list_non_financial = df[(df['iso_country_code'] == country)&(df['is_finance'] == False)]['company_name'].unique().tolist()
  if (len(company_list_financial) < 1) or (len(company_list_non_financial) < 1):
    return None
  else:
    return get_decorr_two_group_companies_zhou(start,end,company_list_financial,company_list_non_financial)
@jit
def get_country_corr_matrix(start,end,prob_sub=0.9):
  countries = get_country_codes()
  mat = pd.DataFrame({'country':countries,'country_corr':[0 for i in range(len(countries))]})
  for i in range(len(countries)):
    mat.at[i,'country_corr'] = get_country_corr(start,end,get_country_codes()[i])
#     print(mat)
  return mat

C:\Users\ZYWCH\AppData\Local\Temp\ipykernel_9556\1026236306.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [20]:
# get_country_corr('2018-12-31','2021-12-31','ARG')

In [21]:
# try:
#   print(get_country_corr('2018-12-31','2021-12-31','ARG'))
# except:
#   pass

In [28]:
analyse_two_companies_zhou('2022-12-31','2023-12-31',get_companies(),'GRUPO DE INVERSIONES SURAMER', 'MINEROS SA')

(-1.771871617782744e+90, 5.064646195530967e-191, 7.751865664855691e-23)

In [23]:
# get_country_corr('2022-12-31','2023-12-31','COL')

In [24]:
# print(get_country_corr('2022-12-31','2023-12-31','VGB'))

In [31]:
get_country_corr_matrix('2022-12-31','2023-12-31')

C:\Users\ZYWCH\AppData\Local\Temp\ipykernel_9556\1026236306.py:10: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "get_country_corr_matrix" failed type inference due to: Untyped global name 'get_country_codes': Cannot determine Numba type of <class 'function'>

File "..\..\AppData\Local\Temp\ipykernel_9556\1026236306.py", line 12:
<source missing, REPL/exec in use?>

  @jit
C:\Users\ZYWCH\AppData\Local\Temp\ipykernel_9556\1026236306.py:10: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "get_country_corr_matrix" failed type inference due to: Untyped global name 'get_country_codes': Cannot determine Numba type of <class 'function'>

File "..\..\AppData\Local\Temp\ipykernel_9556\1026236306.py", line 12:
<source missing, REPL/exec in use?>

  @jit
c:\Users\ZYWCH\anaconda3\Lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "get_country_corr_matrix" was com

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'get_companies':[0m [1m[1mCannot determine Numba type of <class 'function'>[0m
[1m
File "..\..\AppData\Local\Temp\ipykernel_9556\1026236306.py", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [ ]:
get_country_corr_matrix('2021-12-31','2023-12-31')

88
405
308
24
320
1
2


,country,country_corr
0,ARG,0.395769
1,PER,NaN
2,CUW,NaN
3,BRA,0.042270
4,CHL,0.005728
5,COL,0.017695
6,JAM,NaN
7,CYM,NaN
8,MEX,0.023148
9,PAN,0.394991


In [ ]:
get_country_corr_matrix('2018-12-31','2023-12-31')

88
405
308


In [ ]:
get_country_corr_matrix('2006-12-31','2023-12-31')

# Calculate the financial sector coefficient


In [ ]:
list(filter(lambda x: x[0]!=x[1],list(itertools.combinations(['a','b','c','d'],2))))

In [ ]:
def get_finance_corr(start,end,country,prob_sub=0.9):
  companies = get_companies()
  company_list_financial = df[(df['iso_country_code'] == country)&(df['is_finance'] == True)]['company_name'].unique().tolist()
  if (len(company_list_financial) <= 1):
    return None
  else:
    return get_decorr_two_group_companies_zhou(start,end,company_list_financial,company_list_financial)

In [ ]:
def get_finance_corr_matrix(start,end,prob_sub=0.9):
  countries = get_country_codes()
  mat = pd.DataFrame({'country':countries,'finance_corr':[0 for i in range(len(countries))]})
  for i in range(len(countries)):
    mat.at[i,'finance_corr'] = get_finance_corr(start,end,get_country_codes()[i])
  return mat

In [ ]:
# get_finance_corr('2018-12-31','2023-12-31','ARG')

In [ ]:
print(get_finance_corr_matrix('2022-12-31','2023-12-31'))

In [ ]:
get_finance_corr_matrix('2021-12-31','2023-12-31'))

In [ ]:
get_finance_corr_matrix('2018-12-31','2023-12-31')

In [ ]:

get_finance_corr_matrix('2013-12-31','2023-12-31')

In [ ]:
get_finance_corr_matrix('2006-12-31','2023-12-31')

# Calculate the non financial sector coefficient


In [ ]:
def get_non_finance_corr(start,end,country,prob_sub=0.9):
  companies = get_companies()
  company_list_non_financial = df[(df['iso_country_code'] == country)&(df['is_finance'] == False)]['company_name'].unique().tolist()
  if (len(company_list_non_financial) <= 1):
    return None
  else:
    return get_decorr_two_group_companies_zhou(start,end,company_list_non_financial,company_list_non_financial)

In [ ]:
def get_non_finance_corr_matrix(start,end,prob_sub=0.9):
  countries = get_country_codes()
  mat = pd.DataFrame({'country':countries,'non_finance_corr':[0 for i in range(len(countries))]})
  for i in range(len(countries)):
    mat.at[i,'non_finance_corr'] = get_non_finance_corr(start,end,get_country_codes()[i])
  return mat

In [ ]:
# get_non_finance_corr('2018-12-31','2023-12-31','COL',prob_sub=0.9)

In [ ]:
get_non_finance_corr_matrix('2022-12-31','2023-12-31')


In [ ]:
get_non_finance_corr_matrix('2021-12-31','2023-12-31')

In [ ]:
get_non_finance_corr_matrix('2018-12-31','2023-12-31')

In [ ]:

get_non_finance_corr_matrix('2006-12-31','2023-12-31')

In [ ]:
# def analyse_two_companies_zhou(start,end,companies,company1,company2,prob_sub=0.9):
#   '''
#   This function returns the default correlation between 2 companies in a given time horizon by using the
#   closed form formula from Zhou(2001)
#   prob_sub is the subsituting probability if one of the marginal probability is greater than 1
#   '''
#   ### get the essential parameter for Zhou's either probability default
#   # print(companies.tolist().index(company1),companies.tolist().index(company2))
#   v1 = get_Vi0(start,company1,company2)[0]
#   v2 = get_Vi0(start,company1,company2)[1]
#   k1 = get_Ki(start,company1,company2)[0]
#   k2 = get_Ki(start,company1,company2)[1]
#   sigma1 = get_sigma_i(company1)
#   sigma2 = get_sigma_i(company2)
#   rho = get_rho(start,end,companies,'Close',company1,company2)
#   t = get_t(start,end)
#   z1 = (math.log(v1/k1))/sigma1
#   z2 = (math.log(v2/k2))/sigma2
#   alpha = math.atan((-math.sqrt(1-rho**2))/rho) if rho < 0 else math.pi +  math.atan((-math.sqrt(1-rho**2))/rho)
#   theta0 = math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2)) if (z2*math.sqrt(1-rho**2))/(z1-rho*z2) > 0 else math.pi + math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2))
#   r0 = z2/math.sin(theta0)
#   inf_sum = 0
#   for n in [i for i in range(1,1000000,2)]:
#     bassel_sum = sp.special.iv(0.5*(((n*math.pi)/alpha)+1),(r0**2/(4*t)))+ sp.special.iv(0.5*(((n*math.pi)/alpha)-1),(r0**2/(4*t)))
#     inf_sum += 1/n * math.sin((n*math.pi*theta0)/alpha)*bassel_sum
#   # print(round(inf_sum,1),round(v1/k1,1),round(v2/k2,1),round(rho,2),t,round(z1,1),round(z2,1),round(alpha,1),round(theta0,1),round(r0,1))
#   ### calculate the marginal default probability and correlation
#   marginal_prob_1 = 2*sp.stats.norm.cdf(-z1/math.sqrt(t))
#   marginal_prob_2 = 2*sp.stats.norm.cdf(-z2/math.sqrt(t))
#   ### store the original value if any of the marginal prob >=1
#   marginal_prob_1_original = marginal_prob_1 if marginal_prob_1 >=1 else marginal_prob_1
#   marginal_prob_2_original = marginal_prob_2 if marginal_prob_2 >=1 else marginal_prob_2
#   ### use the prob_sub to calculate the correlation
#   marginal_prob_1_cal = prob_sub if marginal_prob_1 >=1 else marginal_prob_1
#   marginal_prob_2_cal = prob_sub if marginal_prob_2 >=1 else marginal_prob_2
#   var_1 = marginal_prob_1*(1-marginal_prob_1_cal)
#   var_2 = marginal_prob_2*(1-marginal_prob_2_cal)
#   either_default_prob = 1-(2*r0/(math.sqrt(2*math.pi*t)))*math.exp(-(r0**2/(4*t)))*inf_sum
#   default_corr = (marginal_prob_1_cal+marginal_prob_2_cal-either_default_prob-marginal_prob_1_cal*marginal_prob_2_cal)/(math.sqrt(var_1*var_2))
#   return default_corr,marginal_prob_1_original,marginal_prob_2_original


# def get_decorr_two_industries_zhou(start,end,industry1,industry2,prob_sub,show_non_normal = None):
#   companies = get_companies()
#   company_list1 = get_companies_from_industry(industry1)
#   company_list2 = get_companies_from_industry(industry2)
#   combination_list =  list(itertools.product(company_list1,company_list2))
#   print(len(company_list1),len(company_list2),len(combination_list))
#   num_pairs = len(combination_list)
#   corr_sum = 0
#   not_normal_sum = 0
#   non_normal_dic = {}
#   for (firm1,firm2) in combination_list:
#     result = analyse_two_companies_zhou(start,end,companies,firm1,firm2,prob_sub=0.9)
#     corr = result[0]
#     # print(corr_sum)
#     prob_default_1 = result[1]
#     prob_default_2 = result[2]
#     # print(corr,prob_default_1,prob_default_2)
#     # print(abs(corr)>1)
#     if (abs(corr) > 1):
#       num_pairs -= 1
#       corr_sum += 0
#       non_normal_dic[corr] = [prob_default_1,prob_default_2]
#     else:
#       corr_sum += corr
#   if show_non_normal == 'Yes':
#     return non_normal_dic
#   else:
#     return corr_sum/num_pairs

# def get_decorr_two_industries_zhou(start,end,industry1,industry2,prob_sub=0.9,show_non_normal = None):
#   companies = get_companies()
#   company_list1 = get_companies_from_industry(industry1)
#   company_list2 = get_companies_from_industry(industry2)
#   combination_list =  list(itertools.product(company_list1,company_list2))
#   print(len(company_list1),len(company_list2),len(combination_list))
#   num_pairs = len(combination_list)
#   corr_sum = 0
#   not_normal_sum = 0
#   non_normal_dic = {}
#   for (firm1,firm2) in combination_list:
#     result = analyse_two_companies_zhou(start,end,companies,firm1,firm2,prob_sub)
#     corr = result[0]
#     # print(corr_sum)
#     prob_default_1 = result[1]
#     prob_default_2 = result[2]
#     # print(corr,prob_default_1,prob_default_2)
#     # print(abs(corr)>1)
#     if (abs(corr) > 1):
#       num_pairs -= 1
#       corr_sum += 0
#       non_normal_dic[corr] = [prob_default_1,prob_default_2]
#     else:
#       corr_sum += corr
#   if show_non_normal == 'Yes':
#     return non_normal_dic
#   else:
#     return corr_sum/num_pairs

In [ ]:
# analyse_two_companies_zhou('2018-12-31','2023-12-31',get_companies(),get_companies()[3],get_companies()[4])[0]

In [ ]:
# industry_corr_matrix_5yr = np.zeros((len(get_industries()),len(get_industries())))
# for i in [i for i in range(0,len(get_companies()))]:
#   for j in [j for j in range(0,len(get_companies()))]:
#     if i == j:
#       industry_corr_matrix_5yr[i,j] = 1
#     else:
#       corr_i_j = get_decorr_two_industries_zhou('2018-12-31','2023-12-31',get_industries()[i],get_industries()[j],0.9)
#       industry_corr_matrix_5yr[i,j] = corr_i_j
# industry_corr_df_1 = pd.DataFrame(industry_corr_matrix_5yr)
# industry_corr_df_1

In [ ]:
# dic_1 = get_decorr_two_industries_zhou(get_date()[20],get_date()[24],get_industries()[4],get_industries()[10],0.9,'Yes')
# dic_2 = get_decorr_two_industries_zhou(get_date()[20],get_date()[24],get_industries()[4],get_industries()[10],0.95,'Yes')
# dic_3 = get_decorr_two_industries_zhou(get_date()[20],get_date()[24],get_industries()[4],get_industries()[10],0.99,'Yes')
# corr_list_1 = list(dic_1.keys())
# corr_list_2 = list(dic_2.keys())
# corr_list_3 = list(dic_3.keys())
# x1_list = [i for i in range(len(corr_list_1))]
# x2_list = [i for i in range(len(corr_list_2))]
# x3_list = [i for i in range(len(corr_list_3))]
# plt.plot(x1_list,corr_list_1,label = 'prob_sub = 0.9')
# plt.plot(x2_list,corr_list_2,label = 'prob_sub = 0.95' )
# plt.plot(x3_list,corr_list_3,label = 'prob_sub = 0.99')
# plt.ylabel('Default correlation')
# plt.xlabel('index')
# plt.legend()
# plt.show()

# Reproduce the result from Zhou(2001) paper

In [ ]:
# V1_0 = 1.8
# V2_0 = 1.8
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.2, 20, 0.5).tolist()
# rho_try_list = [0.1,0.2,0.4]
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_0,V2_0,k1,k2,sigma1,sigma2,-0.1,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_0,V2_0,k1,k2,sigma1,sigma2,0.2,t))
#  default_corr_list_3.append(get_default_corr_zhou(V1_0,V2_0,k1,k2,sigma1,sigma2,0.4,t))
# plt.plot(t_try_list,default_corr_list_1,label = 'rho = -0.1')
# # plt.plot(t_try_list,default_corr_list_2,label = 'rho = 0.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'rho = 0.4')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.legend()
# plt.show()

In [ ]:
# bV1_01 = 2
# V2_01 = 2
# V1_02 = 2.5
# V2_02 = 2.5
# V1_03 = 3
# V2_03 = 3
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 20.0, 0.5).tolist()
# rho_try_list = [0.1,0.2,0.4]
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 2.5')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 3')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 1.2
# V2_01 = 1.2
# V1_02 = 1.5
# V2_02 = 1.5
# V1_03 = 2
# V2_03 = 2
# V1_04 = 7
# V2_04 = 7
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.1,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.1,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.1,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.1,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 1.5')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = V2/K2 = 7')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.1')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 1.2
# V2_01 = 1.2
# V1_02 = 1.5
# V2_02 = 1.5
# V1_03 = 2
# V2_03 = 2
# V1_04 = 7
# V2_04 = 7
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 1.5')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = V2/K2 = 7')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 1.2
# V2_01 = 1.2
# V1_02 = 1.5
# V2_02 = 1.5
# V1_03 = 2
# V2_03 = 2
# V1_04 = 7
# V2_04 = 7
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.9,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.9,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.9,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.9,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 1.5')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = V2/K2 = 7')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.9')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 1.2
# V2_01 = 1.2
# V1_02 = 1.5
# V2_02 = 1.5
# V1_03 = 2
# V2_03 = 2
# V1_04 = 7
# V2_04 = 7
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,-0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,-0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,-0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,-0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 1.5')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = V2/K2 = 7')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = -0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 1.2
# V2_01 = 1.2
# V1_02 = 1.2
# V2_02 = 3
# V1_03 = 1.2
# V2_03 = 5
# V1_04 = 1.2
# V2_04 = 7
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 1.2 V2/K2 = 3')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 1.2 V2/K2 = 5')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 1.2 V2/K2 = 7')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.2
# V2_01 = 0.2
# V1_02 = 0.3
# V2_02 = 0.3
# V1_03 = 0.8
# V2_03 = 0.8
# V1_04 = 0.95
# V2_04 = 0.95
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 0.1')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 0.3')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 0.8')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = V2/K2 = 0.95')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.2
# V2_01 = 1.2
# V1_02 = 0.3
# V2_02 = 1.2
# V1_03 = 0.8
# V2_03 = 1.2
# V1_04 = 0.95
# V2_04 = 1.2
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.2, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.3, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.8 V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 0.95 V2/K2 = 1.2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.8
# V2_01 = 1.2
# V1_02 = 0.85
# V2_02 = 1.2
# V1_03 = 0.9
# V2_03 = 1.2
# V1_04 = 0.95
# V2_04 = 1.2
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.8, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.85, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.9 V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 0.95 V2/K2 = 1.2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.8
# V2_01 = 1.05
# V1_02 = 0.85
# V2_02 = 1.05
# V1_03 = 0.9
# V2_03 = 1.05
# V1_04 = 0.95
# V2_04 = 1.05
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.8, V2/K2 = 1.05')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.85, V2/K2 = 1.05')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.9 V2/K2 = 1.05')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 0.95 V2/K2 = 1.05')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.2
# V2_01 = 1.05
# V1_02 = 0.4
# V2_02 = 1.05
# V1_03 = 0.6
# V2_03 = 1.05
# V1_04 = 0.9
# V2_04 = 1.05
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.2, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.4, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.6 V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 0.9 V2/K2 = 1.2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.2
# V2_01 = 1.05
# V1_02 = 0.4
# V2_02 = 1.05
# V1_03 = 0.6
# V2_03 = 1.05
# V1_04 = 0.9
# V2_04 = 1.05
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.2, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.4, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.6 V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 0.9 V2/K2 = 1.2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# k1 = 1
# k2 = 1
# def get_v2_from_v1(v1,diff):
#   return v1+diff
# for i in rnage

In [ ]:
# V1_01 = 0.1
# V2_01 = 2
# V1_02 = 0.3
# V2_02 = 2
# V1_03 = 0.9
# V2_03 = 2
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 20.0, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.1, V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.3, V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.9, V2/K2 = 2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.1
# V2_01 = 2
# V1_02 = 0.3
# V2_02 = 2
# V1_03 = 0.9
# V2_03 = 2
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 20.0, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.1, V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.3, V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.9, V2/K2 = 2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.1
# V2_01 = 1.2
# V1_02 = 0.3
# V2_02 = 1.2
# V1_03 = 0.9
# V2_03 = 1.2
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 20.0, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.1, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.3, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.9, V2/K2 = 1.2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

# Check the avilability of long term liability data

In [ ]:
# isin_list = ['CL0002694637', 'CL0000000035', 'MXP000511016', 'BRABEVACNOR1', 'MX01AM050019', 'BRAMERACNOR6', 'MX01AX040009', 'COB01PA00030', 'BRBBASACNOR3', 'BRBNBRACNOR8', 'PAP169941328', 'BRSANBACNPR5', 'CLP1506A1070', 'BRBRKMACNPA4', 'BRBRFSACNOR8', 'CLP256251073', 'MXP225611567', 'CLP3615W1037', 'BRCEEBACNOR3', 'BRSBSPACNOR5', 'BRCMIGACNPR3', 'BRCSRNACNOR0', 'BRCSNAACNOR6', 'CLP249051044', 'BRCSANACNOR6', 'BMG2519Y1084', 'MXP339881098', 'MXP369181377', 'CLP3697S1034', 'BREMBRACNOR4', 'CLP371151059', 'CL0000001314', 'CLP7847L1080', 'CLP371861061', 'CLP3710M1090', 'CL0001583070', 'CLP3880F1085', 'MXP320321310', 'MX01GC2M0006', 'BRGGBRACNPR8', 'BRGOLLACNPR4', 'MXP4948K1056', 'MXP495211262', 'COT13PA00086', 'MX01KU000012', 'MXP370841019', 'MXP791641105', 'MXP4987V1378', 'MX01ID000009', 'MXP554091415', 'BRMYPKACNOR7', 'MXP606941179', 'CL0000000423', 'BRRENTACNOR4', 'CLP969051075', 'CLP6877L1036', 'BROIBRACNPR8', 'MX01OR010004', 'ARP432631215', 'BRPETRACNPR6', 'CL0002455229', 'BRSUZBACNOR0', 'ARP9028N1016', 'ARP9308R1039', 'BRTUPYACNOR1', 'BRUGPAACNOR8', 'BRUSIMACNPA6', 'BRVALEACNOR0', 'ARP9897X1319']

In [ ]:
# len(isin_list)

In [ ]:
# balance_sheet = pd.read_csv('balance sheet.csv')

In [ ]:
# balance_sheet.head(1)

In [ ]:
# temp = balance_sheet[balance_sheet['isin'].isin(isin_list)]
# temp.shape

In [ ]:
# np.sum(temp.isnull(), axis = 0)

In [ ]:
# temp[temp['lltq'].isnull()]['datadate'].value_counts()

In [ ]:
# np.sum(temp[temp['lltq'].isnull()]['datadate'].value_counts())

In [ ]:
# temp_company = temp[temp['lltq'].isnull()]['isin'].value_counts()
# temp[temp['lltq'].isnull()]['isin'].value_counts()

In [ ]:
# np.sum(temp[temp['lltq'].isnull()]['isin'].value_counts())